In [2]:
#Financial Computing HW4
import numpy as np
from scipy.stats import norm

#input values setting
st = 50
r = 0.1
q = 0
sigma = 0.4
T = 0.25
t = 0

In [11]:
#CRR for European put
def Lookback_European_put(st, r, q, sigma, T, t, n, smax):
    dt = (T-t)/n
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    p = (np.exp((r-q)*dt)-d)/(u-d)
    s = np.zeros((n+1, n+1))
    s_max = np.zeros((n+1, n+1, n+1+n+1)) #Store all the possible Smax of each node
    put = np.zeros((n+1, n+1, n+1+n+1))
    num_smax = np.zeros((n+1, n+1)) #The counts of Smax in each node
    s[0, 0] = st
    s_max[0, 0, 0] = st if st>smax else smax
    num_smax[0, 0] += 1


    #The stock prices of the last two columns
    for i in range(0, n+1):
        for j in range(n-1, n+1):
            if i <= j:
                s[i, j] = st*(u**(j-i))*(d**i)
    if n%2 == 0:
        s[int(n/2), n] = st
    else:
        s[int((n-1)/2), n-1] = st

    #Avoid the computing error
    for j in range(n-2, 0, -1):
        for i in range(0, j+1):
            s[i, j] = s[i+1, j+2]

    #Compute the Smax
    #Diagonal: the only Smax is Smax of s[0, 0]
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == j:
                s_max[i, j, 0] = s_max[0, 0, 0]
                num_smax[i, j] += 1

    #The first row: the only Smax is itself
    for j in range(1, n+1):
        s_max[0, j, 0] = s[0, j] if s[0, j] > smax else smax
        num_smax[0, j] += 1

    #Inherit Smax form parents
    for j in range(2, n+1):
        for i in range(1, j):
            for k in range(int(num_smax[i-1, j-1])):
                if s_max[i-1, j-1, k] >= s[i, j] and s_max[i-1, j-1, k] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s_max[i-1, j-1, k]
                    num_smax[i, j] += 1
                elif s_max[i-1, j-1, k] < s[i, j] and s[i, j] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s[i, j]
                    num_smax[i, j] += 1

    for j in range(2, n+1):
        for i in range(1, j):
            for k in range(int(num_smax[i, j-1])):
                if s_max[i, j-1, k] >= s[i, j] and s_max[i, j-1, k] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s_max[i, j-1, k]
                    num_smax[i, j] += 1
                elif s_max[i, j-1, k] < s[i, j] and s[i, j] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s[i, j]
                    num_smax[i, j] += 1

    #The put value of the last column
    for i in range(0, n+1):
        for k in range(int(num_smax[i, n])):
            put[i, n, k] = max(0, s_max[i, n, k]-s[i, n])

    for j in range(n-1, -1, -1):
        for i in range(0, j+1):
            for k in range(int(num_smax[i, j])):
                for l in range(int(num_smax[i, j+1])):
                    if s_max[i, j, k] == s_max[i, j+1, l] :
                        put_u = put[i, j+1, l]
                    elif s_max[i, j, k] < s_max[i, j+1, int(num_smax[i, j+1])-1]:
                        put_u = put[i, j+1, int(num_smax[i, j+1])-1]
                for m in range(int(num_smax[i+1, j+1])):
                    if s_max[i, j, k] == s_max[i+1, j+1, m]:
                        put_d = put[i+1, j+1, m]
                put[i, j, k] = np.exp(-r*dt)*(p*put_u+(1-p)*put_d)
    return put[0, 0, 0]

print('n=100, Smax,t = 50: the European lookback put price is', Lookback_European_put(st, r, q, sigma, T, t, 100, 50))
print('n=100, Smax,t = 60: the European lookback put price is', Lookback_European_put(st, r, q, sigma, T, t, 100, 60))
print('n=100, Smax,t = 70: the European lookback put price is', Lookback_European_put(st, r, q, sigma, T, t, 100, 70))
       

n=100, Smax,t = 50: the European lookback put price is 7.236947653488883
n=100, Smax,t = 60: the European lookback put price is 10.915769511478048
n=100, Smax,t = 70: the European lookback put price is 18.79461070112326


In [18]:
#CRR for American put
def Lookback_American_put(st, r, q, sigma, T, t, n, smax):
    dt = (T-t)/n
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    p = (np.exp((r-q)*dt)-d)/(u-d)
    s = np.zeros((n+1, n+1))
    s_max = np.zeros((n+1, n+1, n+1+n+1)) #Store all the possible Smax of each node
    put_american = np.zeros((n+1, n+1, n+1+n+1))
    num_smax = np.zeros((n+1, n+1)) #The counts of Smax in each node
    s[0, 0] = st
    s_max[0, 0, 0] = st if st>smax else smax
    num_smax[0, 0] += 1

    #The stock prices of the last two columns
    for i in range(0, n+1):
        for j in range(n-1, n+1):
            if i <= j:
                s[i, j] = st*(u**(j-i))*(d**i)
    if n%2 == 0:
        s[int(n/2), n] = st
    else:
        s[int((n-1)/2), n-1] = st

    #Avoid the computing error
    for j in range(n-2, 0, -1):
        for i in range(0, j+1):
            s[i, j] = s[i+1, j+2]

    #Compute the Smax
    #Diagonal: the only Smax is Smax of s[0, 0]
    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == j:
                s_max[i, j, 0] = s_max[0, 0, 0]
                num_smax[i, j] += 1

    #The first row: the only Smax is itself
    for j in range(1, n+1):
        s_max[0, j, 0] = s[0, j] if s[0, j] > smax else smax
        num_smax[0, j] += 1

    #Inherit Smax form parents
    for j in range(2, n+1):
        for i in range(1, j):
            for k in range(int(num_smax[i-1, j-1])):
                if s_max[i-1, j-1, k] >= s[i, j] and s_max[i-1, j-1, k] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s_max[i-1, j-1, k]
                    num_smax[i, j] += 1
                elif s_max[i-1, j-1, k] < s[i, j] and s[i, j] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s[i, j]
                    num_smax[i, j] += 1

    for j in range(2, n+1):
        for i in range(1, j):
            for k in range(int(num_smax[i, j-1])):
                if s_max[i, j-1, k] >= s[i, j] and s_max[i, j-1, k] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s_max[i, j-1, k]
                    num_smax[i, j] += 1
                elif s_max[i, j-1, k] < s[i, j] and s[i, j] not in s_max[i, j]:
                    s_max[i, j, int(num_smax[i, j])] = s[i, j]
                    num_smax[i, j] += 1

    #The put value of the last column
    for i in range(0, n+1):
        for k in range(int(num_smax[i, n])):
            put_american[i, n, k] = max(0, s_max[i, n, k]-s[i, n])

    for j in range(n-1, -1, -1):
        for i in range(0, j+1):
            for k in range(int(num_smax[i, j])):
                for l in range(int(num_smax[i, j+1])):
                    if s_max[i, j, k] == s_max[i, j+1, l] :
                        put_u = put_american[i, j+1, l]
                    elif s_max[i, j, k] < s_max[i, j+1, int(num_smax[i, j+1])-1]:
                        put_u = put_american[i, j+1, int(num_smax[i, j+1])-1]
                for m in range(int(num_smax[i+1, j+1])):
                    if s_max[i, j, k] == s_max[i+1, j+1, m]:
                        put_d = put_american[i+1, j+1, m]
                put_american[i, j, k] = np.exp(-r*dt)*(p*put_u+(1-p)*put_d)
                put_american[i, j, k] = max(put_american[i, j, k], s_max[i, j, k]-s[i, j])

    return put_american[0, 0, 0]

print('n=100, Smax,t = 50: the American lookback put price is', Lookback_American_put(st, r, q, sigma, T, t, 100, 50))
print('n=100, Smax,t = 60: the American lookback put price is', Lookback_American_put(st, r, q, sigma, T, t, 100, 60))
print('n=100, Smax,t = 70: the American lookback put price is', Lookback_American_put(st, r, q, sigma, T, t, 100, 70))

n=100, Smax,t = 50: the American lookback put price is 7.439574213907955
n=100, Smax,t = 60: the American lookback put price is 11.37302176523936
n=100, Smax,t = 70: the American lookback put price is 20.0


In [17]:
#Monte Carlo Simulation
st = 50
r = 0.1
q = 0
sigma = 0.4
T = 0.25
t = 0
num_simulations = 10000
num_repetitions = 20

def Lookback_European_put_MC(st, r, q, sigma, T, t, n, smax, num_simulations, num_repetitions):
    dt = (T-t)/n
    put_lookback = []
    for i in range(num_repetitions):
        put = []
        for j in range(num_simulations):
            count = 0
            st_ = [st]
            while count<n:
                z = norm.rvs(0, 1)
                st_.append(np.exp(z*sigma*np.sqrt(dt) + np.log(st_[count])+(r-q-0.5*sigma*sigma)*dt))
                count += 1
            put.append(max(max(st_), smax)-st_[-1])
        put_lookback.append(np.exp(-r*T)*np.mean(put))

    return np.mean(put_lookback), np.mean(put_lookback)+2*np.std(put_lookback), np.mean(put_lookback)-2*np.std(put_lookback)
    
print('n = 100, Smax,t = 50:')
print('the price of European lookback put and the 95% confidence interval are', 
      Lookback_European_put_MC(st, r, q, sigma, T, t, 100, 50, num_simulations, num_repetitions)) 
    
print('n = 100, Smax,t = 60:')
print('the price of European lookback put and the 95% confidence interval are', 
      Lookback_European_put_MC(st, r, q, sigma, T, t, 100, 60, num_simulations, num_repetitions)) 

print('n = 100, Smax,t = 70:')
print('the price of European lookback put and the 95% confidence interval are', 
      Lookback_European_put_MC(st, r, q, sigma, T, t, 100, 70, num_simulations, num_repetitions)) 


n = 100, Smax,t = 50:
the price of European lookback put and the 95% confidence interval are (7.137242872179982, 7.2503937770416975, 7.024091967318267)
n = 100, Smax,t = 60:
the price of European lookback put and the 95% confidence interval are (10.884691345495616, 11.018474394431118, 10.750908296560114)
n = 100, Smax,t = 70:
the price of European lookback put and the 95% confidence interval are (18.82325022533044, 18.980353598443035, 18.666146852217846)
